In [ ]:
import sys

sys.path.append('../')

In [ ]:
import logging
import time

import jax.numpy as jnp
import pandas as pd

from src.algorithms.parallel_jax_pso import parallel_jax_pso
from src.config import config

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

In [ ]:
def benchmark_algorithm(
    algorithm_name: str,
    algorithm_fn: callable,
    objective_fn: callable,
    bounds: tuple,
    params: dict,
    num_runs: int,
    num_subswarms: int,
) -> float:
    execution_times = []

    for i in range(num_runs):
        logger.info(
            '%s | Run %d/%d | [RUNNING]',
            algorithm_name,
            i + 1,
            num_runs,
        )

        if algorithm_name == 'JAX PSO':
            if i == 0:
                parallel_jax_pso(objective_fn, bounds, params, num_subswarms)    
            start_time = time.perf_counter()
            parallel_jax_pso(objective_fn, bounds, params, num_subswarms)
        else:
            start_time = time.perf_counter()
            algorithm_fn(objective_fn, bounds, **params)

        elapsed = time.perf_counter() - start_time
        execution_times.append(elapsed)

        logger.info(
            '%s | Run %d/%d | [DONE] in %.6fs',
            algorithm_name,
            i + 1,
            num_runs,
            elapsed,
        )

    return execution_times

def compare_pso_implementations(
    benchmarks: dict,
    algorithms: dict,
    dims: list,
    num_runs: int,
    num_subswarms: int,
    params: dict,
) -> dict:

    logger.info('='*60)
    logger.info(
        'INITIATING BENCHMARKS | Runs: %d | Subswarms: %d',
        num_runs,
        num_subswarms,
    )
    logger.info('='*60)

    results = {
        dim: {
            benchmark_name: {
                algorithm_name: [] for algorithm_name in algorithms
            } for benchmark_name in benchmarks
        } for dim in dims
    }

    for dim in dims:
        params['num_dims'] = dim

        for benchmark_name, (objective_fn, bounds) in benchmarks.items():
            for name, algorithm in algorithms.items():
                logger.info(
                    '>>> Dimension: %d | Benchmark: %s | Algorithm: %s',
                    dim,
                    benchmark_name,
                    name,
                )

                results[dim][benchmark_name][name] = benchmark_algorithm(
                    name,
                    algorithm,
                    objective_fn,
                    bounds,
                    params,
                    num_runs,
                    num_subswarms,
                )

    logger.info('='*60)
    logger.info('COMPLETED | All benchmarks finished successfully')
    logger.info('='*60)
    return results

def save_results(results: dict) -> None:
    rows = []

    for dim, benchmarks_data in results.items():
        for benchmark_name, algorithms_data in benchmarks_data.items():
            for algorithm_name, execution_times in algorithms_data.items():
                mean_time = float(jnp.mean(jnp.array(execution_times)))
                std_time = float(jnp.std(jnp.array(execution_times)))

                rows.extend(
                    [
                        {
                            'Dimension': dim,
                            'Benchmark': benchmark_name,
                            'Algorithm': algorithm_name,
                            'Execution Times': execution_times,
                            'Mean of Execution Times (s)': mean_time,
                            'Standard Deviation of Execution Times (s)': std_time,
                        },
                    ],
                )

    df = pd.DataFrame(rows)
    df.to_csv('../data/results.csv')

In [ ]:
results = compare_pso_implementations(
    config['benchmarks'],
    config['algorithms'],
    config['dims'],
    config['num_runs'],
    config['num_subswarms'],
    config['params'],
)

In [ ]:
save_results(results)